In [1]:
# Домашнее задание:
# Условие: Найти самую длинную последовательность упорядоченных чисел в RDD и вывести её

In [4]:
import pyspark

In [5]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [15]:
from pprint import pprint

In [73]:
if 'sc' in locals():
    sc.stop()
sc = SparkContext("local", "Ordered_sequence")
data = [30, 150, 180, -10, 56, 34, 24, 45, 67, 123, 150, 44, 32, 66, 77, 88]
rdd = sc.parallelize(data)
rdd_i = rdd.zipWithIndex().map(lambda x: (x[1], x[0]))
rdd_j = rdd_i.leftOuterJoin(rdd_i.map(lambda x: (x[0] + 1, x[1]))).sortByKey().mapValues(lambda v: (v[0], v[1], 0 if (v[1] is None) or (v[0] > v[1]) else 1))
pprint(rdd_j.collect())
# Часть когда для каждой последовательности присваивается отдельный индекс, я не смог разобраться, как это сделать в Spark
i = 0
data = []
for row in rdd_j.collect():
    i += row[1][2]
    data.append((i, row[1][0]))
pprint(data)
# Далее снова все делаем на Spark
rdd = sc.parallelize(data)
# посчитаем сколько нужно партиций. Хардкод конечно, но не удалось найти, как разделить RDD на партиции без указания их числа, только по значению ключа
num_partitions = rdd.keys().distinct().count()
# разделим на RDD на партиции и выделим их в явном виде
rdd_part = rdd.partitionBy(num_partitions ,lambda k: k).glom()
longest_seq = rdd_part.map(lambda x: tuple(el[1] for el in x)).max(key=lambda x: len(x))
print(f'Самая длинная последовательность : {longest_seq}')
sc.stop()

[(0, (30, None, 0)),
 (1, (150, 30, 0)),
 (2, (180, 150, 0)),
 (3, (-10, 180, 1)),
 (4, (56, -10, 0)),
 (5, (34, 56, 1)),
 (6, (24, 34, 1)),
 (7, (45, 24, 0)),
 (8, (67, 45, 0)),
 (9, (123, 67, 0)),
 (10, (150, 123, 0)),
 (11, (44, 150, 1)),
 (12, (32, 44, 1)),
 (13, (66, 32, 0)),
 (14, (77, 66, 0)),
 (15, (88, 77, 0))]
[(0, 30),
 (0, 150),
 (0, 180),
 (1, -10),
 (1, 56),
 (2, 34),
 (3, 24),
 (3, 45),
 (3, 67),
 (3, 123),
 (3, 150),
 (4, 44),
 (5, 32),
 (5, 66),
 (5, 77),
 (5, 88)]
Самая длинная последовательность : (24, 45, 67, 123, 150)
